In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import torch 
import torch.nn as nn
from torch.autograd import Variable

import pandas as pd

from sklearn.utils import shuffle
from torchsummary import summary

In [9]:
df= pd.read_csv("/content/diabetes.csv")
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:

from sklearn.model_selection  import train_test_split
X = df.iloc[:,0:-1]
y = df['Age']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train=X_train.values
y_train=y_train.values
X_test=X_test.values
y_test=y_test.values

l = []
for i in y_train:
  if(i>=1 and i<= 8):
    l.append(0)
  elif(i==9 or i==10):
    l.append(1)
  else:
    l.append(2)
y_train2 = np.array(l)
l =[]
for i in y_test:
  if(i>=1 and i<= 8):
    l.append(0)
  elif(i==9 or i==10):
    l.append(1)
  else:
    l.append(2)
y_test2 = np.array(l)

def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()


batch_size = 32 
num_epochs = 100 
learning_rate = 0.01
size_hidden_1 = 100 
num_classes = 30

  
batch_no = len(X_train) // batch_size  
cols = X_train.shape[1] 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Net(torch.nn.Module):
    def __init__(self, num_inputs, size_hidden_1, n_output):
        super(Net, self).__init__()
        self.hidden_layer_1 = torch.nn.Linear(num_inputs, size_hidden_1)   
        self.activation_1 = torch.nn.Tanh() 
        self.output_layer = torch.nn.Linear(size_hidden_1, n_output)   
        self.output_act = torch.nn.Sigmoid()
    def forward(self, x):
        x1 = self.activation_1(self.hidden_layer_1(x))     
        x = self.output_act(self.output_layer(x1))                    
        return x
n_classes2=3
net = Net(cols, size_hidden_1, n_classes2)


(537, 8) (231, 8) (537,) (231,)


In [11]:

#Q (2)

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
loss_func = torch.nn.CrossEntropyLoss()
X_train_copy = X_train

for epoch in range(num_epochs):
    X_train, y_train2 = shuffle(X_train, y_train2)
    train_acc = 0.0
    running_loss = 0.0
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(X_train[start:end]))
        labels = Variable(torch.LongTensor(y_train2[start:end]))
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        acc = get_accuracy(outputs, labels, batch_size)
        train_acc += acc
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
           %(epoch+1, running_loss / (i+1), train_acc/(i+1))) 

Epoch: 1 | Loss: 0.9395 | Train Accuracy: 81.05
Epoch: 2 | Loss: 0.7459 | Train Accuracy: 100.00
Epoch: 3 | Loss: 0.6749 | Train Accuracy: 100.00
Epoch: 4 | Loss: 0.6404 | Train Accuracy: 100.00
Epoch: 5 | Loss: 0.6203 | Train Accuracy: 100.00
Epoch: 6 | Loss: 0.6073 | Train Accuracy: 100.00
Epoch: 7 | Loss: 0.5985 | Train Accuracy: 100.00
Epoch: 8 | Loss: 0.5919 | Train Accuracy: 100.00
Epoch: 9 | Loss: 0.5868 | Train Accuracy: 100.00
Epoch: 10 | Loss: 0.5825 | Train Accuracy: 100.00
Epoch: 11 | Loss: 0.5798 | Train Accuracy: 100.00
Epoch: 12 | Loss: 0.5773 | Train Accuracy: 100.00
Epoch: 13 | Loss: 0.5752 | Train Accuracy: 100.00
Epoch: 14 | Loss: 0.5734 | Train Accuracy: 100.00
Epoch: 15 | Loss: 0.5718 | Train Accuracy: 100.00
Epoch: 16 | Loss: 0.5705 | Train Accuracy: 100.00
Epoch: 17 | Loss: 0.5692 | Train Accuracy: 100.00
Epoch: 18 | Loss: 0.5682 | Train Accuracy: 100.00
Epoch: 19 | Loss: 0.5672 | Train Accuracy: 100.00
Epoch: 20 | Loss: 0.5664 | Train Accuracy: 100.00
Epoch: 21 

In [12]:
#Q(3).

for epoch in range(num_epochs):
    
    X_train, y_train2 = shuffle(X_train, y_train2)
    train_acc = 0.0
    running_loss = 0.0
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(X_train[start:end]))
        labels = Variable(torch.LongTensor(y_train2[start:end]))        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_func(outputs, labels)        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        acc = get_accuracy(outputs, labels, batch_size)
        train_acc += acc
rmse = np.mean(running_loss)          
print(rmse)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score
lr = LinearRegression()
lr.fit(X_train , y_train)
y_pred = lr.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
print(rmse)

8.845421493053436
11.880263696158151
